# TM10007 Assignment template

In [13]:
# Run this to use from colab environment
!git clone https://github.com/Doesjka/TM10007_ML_g9.git

fatal: destination path 'TM10007_ML_g9' already exists and is not an empty directory.


## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [38]:
import os
import pandas as pd

data = pd.read_csv('/content/TM10007_ML_g9/worclipo/Lipo_radiomicFeatures.csv', index_col=0)
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 115
The number of columns: 494


In [63]:
zeros = (data == 0).sum()
threshold = 0.5 * len(data.index)
feature_del = zeros[zeros > 60]

new_data = data.drop(columns=feature_del.index)
print(f'{len(data.columns)-len(new_data.columns)} features were deleted')



23 features were deleted


In [64]:
more_zeros = (new_data == 0).sum()
pd.set_option('display.max_rows', None)
columns_zeros = more_zeros[more_zeros > 0].index
print(f'Of the remaining features, {len(columns_zeros)} features have at least one zero')

for column in columns_zeros:
  new_data[column] = new_data[column].replace(0, new_data[column].mean())


Of the remaining features, 13 features have at least one zero
